In [57]:
import keras
from keras.datasets import cifar10

In [58]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [59]:
print(x_train[0].shape)
# print(x_test[0])

(32, 32, 3)


In [60]:
import numpy as np 
import itertools
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.neighbors import BallTree
from sklearn.metrics import accuracy_score
import pickle
import glob
import cv2

In [61]:
def desSURF(image):
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(image,None)
    #draw keypoints
    #import matplotlib.pyplot as plt		
    #img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
    #plt.imshow(img2),plt.show()
    return kp,des

def describeORB( image):
    #An efficient alternative to SIFT or SURF
    #doc http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_orb/py_orb.html
    #ORB is basically a fusion of FAST keypoint detector and BRIEF descriptor 
    #with many modifications to enhance the performance
    orb=cv2.ORB_create()
    kp, des=orb.detectAndCompute(image,None)
    return kp,des

In [62]:
def getDescriptors(images) : 
    descriptors = []
    
    for image in images: 
        image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = desSURF(image)
        if des is not None : 
            descriptors.append(des)
            
#     descriptors = list(itertools.chain.from_iterable(descriptors))
    descriptors = np.concatenate(descriptors, axis=0)
#     descriptors = np.asarray(descriptors)
    print(descriptors)
    return descriptors

def getVLADDescriptors(images, images_lables, visualDic):
    descriptors = []
    labels = []
    
    count = 0
    for image in images : 
        image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = desSURF(image)
        if des is not None : 
            v = VLAD(des, visualDic)
            print(v)
            descriptors.append(v)
            labels.append(images_lables[count])
        count += 1
            
            
#     descriptors = list(itertools.chain.from_iterable(descriptors))
#     descriptors = np.asarray(descrip/tors)
    print(descriptors)  
    return descriptors, labels
    
def kMeans(training, k) : 
    est = KMeans(n_clusters = k, init = 'k-means++').fit(training)
    return est

def VLAD(X, visualDictionary) : 
    
    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels = visualDictionary.labels_
    k = visualDictionary.n_clusters
    
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V


In [ ]:
sift_des = getDescriptors(np.concatenate((x_train, x_test), axis = 0))
# visDic = kMeans(sift_des, 50)



In [39]:
visDic = MiniBatchKMeans(init='k-means++', n_clusters=50,max_iter=1000, batch_size=1000,
                  n_init=10, max_no_improvement=10, verbose=0).fit(sift_des)

In [47]:
vlad_des, labels = getVLADDescriptors(x_train, y_train, visDic)
print ("Hola")
vlad_des_test, labels_test = getVLADDescriptors(x_test, y_test, visDic)

[ 0.          0.          0.         ...  0.00402814 -0.00300329
  0.00245246]
[ 0.          0.          0.         ...  0.00227313 -0.00166725
 -0.00369001]
[ 0.00337464  0.00654711 -0.00564341 ... -0.00606835 -0.01244579
 -0.00740522]
[ 0.0113909   0.01289919 -0.00948215 ... -0.00166221 -0.0041023
 -0.00240682]
[-0.00485049 -0.00582987  0.00370811 ...  0.01511167  0.01328507
  0.00252323]
[ 0.00313677  0.0035824  -0.00625016 ...  0.00487102  0.0103457
  0.00405396]
[0.01225034 0.01124078 0.0058857  ... 0.00555091 0.00228235 0.00373886]
[-0.01032174 -0.01327438  0.01496507 ... -0.00953238 -0.00039575
  0.00917794]
[-0.00738224 -0.00411613 -0.0042253  ...  0.00327297 -0.00812303
 -0.00670859]
[ 0.          0.          0.         ... -0.00556778 -0.00410133
 -0.00050838]
[0.01505653 0.00594996 0.00629351 ... 0.         0.         0.        ]
[-0.01032353  0.00331784 -0.00784367 ... -0.01174799 -0.00518778
  0.01037191]
[ 0.          0.          0.         ...  0.00506117 -0.0084969
 -0.

        0.00562258,  0.01073602])]


In [49]:
clf = cv2.ml.KNearest_create()
clf.train(np.float32(vlad_des), cv2.ml.ROW_SAMPLE, np.float32(labels))


True

In [51]:
ret, results, neighbours ,dist = clf.findNearest(np.float32(vlad_des_test), k=10)

In [52]:
print (results)
print (labels_test)

[[1.]
 [0.]
 [7.]
 [0.]
 [1.]
 [3.]
 [3.]
 [3.]
 [2.]
 [3.]]
[array([3]), array([8]), array([8]), array([0]), array([6]), array([6]), array([1]), array([6]), array([3]), array([1])]


In [56]:
pred_label = []

for label in results:
    pred_label.append(int(label))

print (y_test)
print (pred_label)
    
accuracy_score(y_test, pred_label)

[[3]
 [8]
 [8]
 [0]
 [6]
 [6]
 [1]
 [6]
 [3]
 [1]]
[1, 0, 7, 0, 1, 3, 3, 3, 2, 3]


0.1